In [54]:
import torch

model_path = "debug/20230602151850/spectral_fc_01.pth"
dict = torch.load(model_path, map_location=torch.device('cpu'))
print("epoch",dict['epoch'])
for k, v in dict["state_dict"].items():
    if "alpha" in k:
        print(k,v)


epoch 109
module.alpha tensor([ 9.2566,  9.4211,  9.5625, 10.5863, 10.6892, 10.6220, 10.2805,  9.4553,
         9.1071,  9.1877])
